In [137]:
! pip install mysql-connector-python
import mysql.connector
import pandas as pd
import json
import requests
import calendar

In [2]:
con = open('ref_con/db-prima', 'r').read().splitlines()
how = input('how to connect(local/public):')

mydb = mysql.connector.connect(
  host=con[0] if how == 'public' else con[1],
  user=con[2],
  password=con[3],
  database=con[4],
  autocommit=True
)

mycursor = mydb.cursor(dictionary=True)

In [176]:
def exec_to_result(q):
  mycursor.execute(q)
  return mycursor.fetchall()

def get_stock_result(code):
    return exec_to_result("select stok_fisik,right(no_so,3) depo, namaDepo \
from inventori_so_detail inner join masterf_depo on right(no_so,3) = masterf_depo.id \
where no_so like 'so2112%' and kode_obat = '"+code+"' and stok_fisik > 0 order by stok_fisik desc")
    
def get_mutasi(kode,sel):
  h = get_stock_result(kode)
  T = 'P02' if sel < 0 else 'P01'
  t = abs(sel)
  if(sel < 0):
    for i in h:
      if (t >= i['stok_fisik']):
          t -= i['stok_fisik']
          i['target'] = 0
          i['trans'] = T
      else:
          i['target'] = i['stok_fisik'] - t
          t = 0
          i['trans'] = T
  else:
    return
  D = []
  for i in h:
    if i['stok_fisik'] != i['target']:
      D.append({'depo':i['depo'],'old_stok':i['stok_fisik'],'new_stok':i['target'],'mutasi':abs(i['target'] - i['stok_fisik']),'trans':T,\
                'kode':'|'+kode+'|','namaDepo':i['namaDepo']})
  return D

def do_request_transaksi(f,d,so):
  url = 'http://sim.rs/simrs/inventori/laporan_inventori/pub_txt_logfar/'
  myobj = {'tgl_dari': f,'tgl_sampai': d,'kode_so': so}

  x = requests.post(url, data = myobj)

  return x.json()

def get_ket_bast(l):
  url = 'http://sim.rs/devapi/rsp/invpenerimaanfarmasi/penerimaan_bast_bku/'
  myobj = {'po': l}

  x = requests.post(url, data = json.dumps(myobj),headers={'Content-Type': 'application/json', 'Accept':'application/json'})
  
  return x.json()

def get_list_simak_date(y,lim_range = 13):
    r = []
    for i in range(1,lim_range,1):
        e = calendar.monthrange(y, i)[1]
        m = str(i).zfill(2)
        r.append(['01-'+m+'-'+str(y)+' 00:00',str(e)+'-'+m+'-'+str(y)+' 23:59',str(y)[2]+str(y)[3]+m])
    return r

  

In [ ]:
import decimal
R = []
kList = open('ref_con/KODE-SELISIH-2021.txt', 'r').read().splitlines()
sList = open('ref_con/NOMINAL-SELISIH-2021.txt', 'r').read().splitlines()
for i, x in enumerate(kList):
  N = get_mutasi(kList[i],decimal.Decimal(sList[i]))
  if N:
   R = R + N
RDF = pd.DataFrame(R)
RDF

In [15]:
RDF.to_csv("RDF-ALL.csv")

In [19]:
for i in range(0,2333):
  N = get_mutasi(kList[i],decimal.Decimal(sList[i]))
  if not N:
      print(kList[i])


020700653


bast dan simak start here !

In [182]:
arr = []
for i in get_list_simak_date(2022,7):
    arr += do_request_transaksi(i[0],i[1],i[2])['d']


In [183]:
dfS = pd.DataFrame(arr)

In [ ]:

listPO = []
for i in dfS[[11]][(dfS[12] == '|M02|')&(dfS[11].str.contains('PPO'))].drop_duplicates().groupby(by=[11]).count().reset_index().values:
    listPO.append(i[0].replace('|',''))
df_po_bast = pd.DataFrame(get_ket_bast(listPO))
df_po_bast
